In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='', project_access_token='')
pc = project.project_context


In [2]:
# @hidden_cell
import ibm_boto3
import io
import pandas as pd

from ibm_botocore.client import Config

storage_metadata = project.get_storage_metadata()

credentials = {
    'endpoint': storage_metadata['properties']['endpoint_url'],
    'service_id': storage_metadata['properties']['credentials']['editor']['service_id'],
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token',
    'api_key': storage_metadata['properties']['credentials']['editor']['api_key']
}

cos = ibm_boto3.resource("s3",
    ibm_api_key_id='',
    ibm_service_instance_id='',
    config=Config(signature_version="oauth"),
    endpoint_url=credentials['endpoint']
)

file = cos.Object(storage_metadata['properties']['bucket_name'], 'covid.csv').get()
file = file["Body"].read().decode()

buffer = io.StringIO(file)

features = pd.read_csv(filepath_or_buffer = buffer, sep=',')

features.drop(['entry_date', 'date_symptoms', 'id'], axis=1, inplace=True)

def normalize_deaths(x):
    if x == '9999-99-99':
        return 99
    else:
        return 1
    
features['date_died'] = features['date_died'].apply(normalize_deaths)
features.describe()

,sex,patient_type,date_died,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
count,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000
mean,1.506726,1.215165,92.742966,76.562952,1.846262,42.622483,50.400692,2.210633,2.280221,2.265029,2.319628,2.145774,2.410030,2.286157,2.138905,2.283765,2.238360,31.573034,1.728651,76.562864
std,0.499955,0.410937,23.959129,39.058676,0.560939,16.659973,47.501579,5.683523,5.327832,5.334658,5.667381,5.459866,6.489959,5.438405,5.395578,5.393232,5.571901,45.082123,0.658710,39.059060
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,99.000000,97.000000,2.000000,31.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,97.000000
50%,2.000000,1.000000,99.000000,97.000000,2.000000,41.000000,97.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,97.000000
75%,2.000000,1.000000,99.000000,97.000000,2.000000,53.000000,97.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,99.000000,2.000000,97.000000
max,2.000000,2.000000,99.000000,99.000000,99.000000,120.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,99.000000,3.000000,99.000000


In [3]:
# Use numpy to convert to arrays
import numpy as np

labels_icu = np.array(features['icu'])
features_icu = features.drop('icu', axis = 1)

features_icu = np.array(features_icu)

labels_death = np.array(features['date_died'])
features_death = features.drop('date_died', axis = 1)

features_death = np.array(features_death)

In [4]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 20 decision trees
rf_icu = RandomForestRegressor(n_estimators = 20, random_state = 8)

# Train the model on training data
rf_icu.fit(features_icu, labels_icu)

# Instantiate model with 20 decision trees
rf_deaths = RandomForestRegressor(n_estimators = 20, random_state = 8)

# Train the model on training data
rf_deaths.fit(features_death, labels_death)

RandomForestRegressor(n_estimators=20, random_state=8)

In [5]:
import pickle
import zlib

d = { 'model_death': rf_deaths, 'model_icu': rf_icu }
obj = pickle.dumps(d)
obj = zlib.compress(obj)


cos.Object(storage_metadata['properties']['bucket_name'], 'models.zip').put(Body=obj)

TypeError: a bytes-like object is required, not 'dict'